In [9]:
import pandas as pd
import numpy as np

In [10]:
df = pd.read_csv('./../DADOS/dados_combinados.csv')
df.head()

,ID_COMODO,ID_USUARIO,NOME_COMODO,DESCRICAO_x,ID_CONSUMO_MENSAL,MES_REFERENCIA,CONSUMO_MENSAL_KWH,ID_ELETRODOMESTICO,NOME_ELETRODOMESTICO,POTENCIA_WATTS,...,CEP,ID_PONTUACAO,PONTOS_TOTAIS,DATA_ATUALIZACAO,ID_RESIDENCIA,TIPO_RESIDENCIA,QUANTIDADE_PESSOAS,NOME,EMAIL,SENHA
0,1,1,Varanda,CAA modo 1 - arejado,1.0,2024-02-01 00:00:00,540.99,1,Ar-condicionado,1662,...,21897-5349,1,95,2024-11-20 00:00:00,1,Apartamento,3,Ana,ana@exemplo.com,senha3
1,2,2,Quarto,CAA modo 2 - com mAA3veis novos,2.0,2024-03-01 00:00:00,414.30,2,LAAmpada,2850,...,36475-3864,2,16,2024-11-20 00:00:00,2,Casa,1,Carlos,carlos@exemplo.com,senha6
2,3,3,Varanda,CAA modo 3 - arejado,3.0,2024-04-01 00:00:00,668.66,3,FogAAo,900,...,40349-7389,3,35,2024-11-20 00:00:00,3,ChAAcara,4,Fernanda,fernanda@exemplo.com,senha9
3,4,4,Quarto,CAA modo 4 - bem iluminado,4.0,2024-05-01 00:00:00,932.57,4,TV,1152,...,82375-7672,4,10,2024-11-20 00:00:00,4,ChAAcara,2,JoAAo,joAAo@exemplo.com,senha12
4,5,5,Quarto,CAA modo 5 - com mAA3veis novos,5.0,2024-06-01 00:00:00,284.18,5,Geladeira,1427,...,81093-3720,5,77,2024-11-20 00:00:00,5,Apartamento,5,Larissa,larissa@exemplo.com,senha15


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [12]:
def categorize_consumo(consumo):
    if consumo < 200:
        return 'Baixo'
    elif 200 <= consumo <= 500:
        return 'Médio'
    else:
        return 'Alto'

In [14]:
# Aplicar a função para criar uma nova coluna 'CATEGORIA_CONSUMO'
df['CATEGORIA_CONSUMO'] = df['CONSUMO_MENSAL_KWH'].apply(categorize_consumo)
df[['CONSUMO_MENSAL_KWH', 'CATEGORIA_CONSUMO']].head()


,CONSUMO_MENSAL_KWH,CATEGORIA_CONSUMO
0,540.99,Alto
1,414.30,Médio
2,668.66,Alto
3,932.57,Alto
4,284.18,Médio


In [15]:
X = df[['POTENCIA_WATTS', 'HORAS_USO_DIA', 'QUANTIDADE_PESSOAS', 'TIPO_RESIDENCIA']]
y = df['CATEGORIA_CONSUMO']

In [16]:
# Vamos usar o ColumnTransformer para aplicar OneHotEncoder apenas para a coluna categórica 'TIPO_RESIDENCIA'
preprocessor = ColumnTransformer(
    transformers=[
        ('residencia', OneHotEncoder(), ['TIPO_RESIDENCIA']),
        ('scaler', StandardScaler(), ['POTENCIA_WATTS', 'HORAS_USO_DIA', 'QUANTIDADE_PESSOAS'])
    ])

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar o pipeline para combinar o pré-processamento e o modelo de classificação
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

In [18]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('residencia', OneHotEncoder(),
                                                  ['TIPO_RESIDENCIA']),
                                                 ('scaler', StandardScaler(),
                                                  ['POTENCIA_WATTS',
                                                   'HORAS_USO_DIA',
                                                   'QUANTIDADE_PESSOAS'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [19]:
y_pred = model.predict(X_test)

In [20]:
print(f'Acurácia: {accuracy_score(y_test, y_pred):.2f}')
print('Relatório de Classificação:')
print(classification_report(y_test, y_pred))
print('Matriz de Confusão:')
print(confusion_matrix(y_test, y_pred))

Acurácia: 0.50
Relatório de Classificação:
              precision    recall  f1-score   support

        Alto       0.62      0.71      0.67         7
       Baixo       0.00      0.00      0.00         1
       Médio       0.00      0.00      0.00         2

    accuracy                           0.50        10
   macro avg       0.21      0.24      0.22        10
weighted avg       0.44      0.50      0.47        10

Matriz de Confusão:
[[5 0 2]
 [1 0 0]
 [2 0 0]]


C:\Users\zenet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\zenet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\zenet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classificati